In [11]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

from my_schema import SCHEMA


OSM_PATH = "CSsample2.xml"
file_in = open("CSsample2.xml")
root = ET.parse("CSsample2.xml").getroot()
schema_file = open("my_schema.py")

def find_element():
    element = []
    for item in root.find("node"):
        element = ET.dump(item)
        return element

element = find_element()

In [5]:
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

SCHEMA = SCHEMA

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

In [11]:
##Run only once
'''Create smaller, sample file for running tests'''
'''
OSM_FILE = "CSMap2"
SAMPLE_FILE = "CSsample2"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write(('<?xml version="1.0" encoding="UTF-8"?>\n').encode())
    output.write(('<osm>\n  ').encode())

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write(('</osm>').encode())'''

In [12]:
'''helper functions for tag assignments in shape_element function'''
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, bytes) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

In [13]:
def create_key(key_string):
    '''
    find "k" values and assign seperate keys and tag types
    '''
    if ":" in key_string:
        new_string = key_string.find(":")
        tag_type = key_string[:new_string]
        new_key = key_string[new_string+1:]
        return [new_key, tag_type]
        
    else:
        new_key = key_string
        tag_type = "regular"
        return [new_key, tag_type]

In [14]:
'''Find secondary tags and add them to tags list of dictionaries'''
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""
    """element is a node or way"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []
    if element.tag == 'node':
        for childName, childValue in element.attrib.items():
            if childName in NODE_FIELDS:
                node_attribs[childName] = childValue
            
        for child in element.iter('tag'):    
            dict = {}
            if PROBLEMCHARS.search(child.attrib['k']):
                continue
            else:
                dict['id'] = element.attrib['id']
                dict['key'] = create_key(child.attrib['k'])[0]
                dict['type'] = create_key(child.attrib['k'])[1]
                dict['value'] = child.attrib['v']
            tags.append(dict)
        
        return {'node': node_attribs, 'node_tags': tags}
        
    elif element.tag == 'way':
        
        for childName, childValue in element.attrib.items():
            if childName in WAY_FIELDS:
                way_attribs[childName] = childValue
                
        '''Process way tags like node tags'''
        for child in element.iter('tag'):
            dict = {}
            if PROBLEMCHARS.search(child.attrib['k']):
                continue
            else:
                dict['id'] = element.attrib['id']
                dict['key'] = create_key(child.attrib['k'])[0]
                dict['type'] = create_key(child.attrib['k'])[1]
                dict['value'] = child.attrib['v']
            tags.append(dict)
        print(tags)
        
        for num, child in enumerate(element.iter('nd')):
            dict = {}
            dict['id'] = element.attrib['id']
            dict['node_id'] = child.attrib['ref']
            dict['position'] = num
            way_nodes.append(dict)
            
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

In [15]:
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "node_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "way_nodes.csv"
WAY_TAGS_PATH = "way_tags.csv"

def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()                
            
        validator = cerberus.Validator()
        
        
        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])
                    
schema_file.close()

In [17]:
if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)

[{'id': '7669789', 'key': 'highway', 'type': 'regular', 'value': 'residential'}, {'id': '7669789', 'key': 'name', 'type': 'regular', 'value': 'Holli Springs Lane'}, {'id': '7669789', 'key': 'cfcc', 'type': 'tiger', 'value': 'A41'}, {'id': '7669789', 'key': 'county', 'type': 'tiger', 'value': 'El Paso, CO'}, {'id': '7669789', 'key': 'name_base', 'type': 'tiger', 'value': 'Holli Springs'}, {'id': '7669789', 'key': 'name_type', 'type': 'tiger', 'value': 'Ln'}, {'id': '7669789', 'key': 'reviewed', 'type': 'tiger', 'value': 'no'}]
[{'id': '7670166', 'key': 'bicycle', 'type': 'regular', 'value': 'no'}, {'id': '7670166', 'key': 'destination', 'type': 'regular', 'value': 'I 25'}, {'id': '7670166', 'key': 'highway', 'type': 'regular', 'value': 'motorway_link'}, {'id': '7670166', 'key': 'oneway', 'type': 'regular', 'value': 'yes'}, {'id': '7670166', 'key': 'cfcc', 'type': 'tiger', 'value': 'A63'}, {'id': '7670166', 'key': 'county', 'type': 'tiger', 'value': 'El Paso, CO'}, {'id': '7670166', 'key

[{'id': '7676075', 'key': 'highway', 'type': 'regular', 'value': 'residential'}, {'id': '7676075', 'key': 'name', 'type': 'regular', 'value': 'Boxelder Drive'}, {'id': '7676075', 'key': 'cfcc', 'type': 'tiger', 'value': 'A41'}, {'id': '7676075', 'key': 'county', 'type': 'tiger', 'value': 'El Paso, CO'}]
[{'id': '7676181', 'key': 'highway', 'type': 'regular', 'value': 'residential'}, {'id': '7676181', 'key': 'maxspeed', 'type': 'regular', 'value': '25 mph'}, {'id': '7676181', 'key': 'name', 'type': 'regular', 'value': 'Suntide Place'}, {'id': '7676181', 'key': 'cfcc', 'type': 'tiger', 'value': 'A41'}, {'id': '7676181', 'key': 'county', 'type': 'tiger', 'value': 'El Paso, CO'}, {'id': '7676181', 'key': 'name_base', 'type': 'tiger', 'value': 'Suntide'}, {'id': '7676181', 'key': 'name_type', 'type': 'tiger', 'value': 'Pl'}, {'id': '7676181', 'key': 'reviewed', 'type': 'tiger', 'value': 'no'}]
[{'id': '7676312', 'key': 'highway', 'type': 'regular', 'value': 'residential'}, {'id': '7676312',

[{'id': '7679953', 'key': 'highway', 'type': 'regular', 'value': 'residential'}, {'id': '7679953', 'key': 'name', 'type': 'regular', 'value': 'Pamlico Drive'}, {'id': '7679953', 'key': 'surface', 'type': 'regular', 'value': 'asphalt'}, {'id': '7679953', 'key': 'cfcc', 'type': 'tiger', 'value': 'A41'}, {'id': '7679953', 'key': 'county', 'type': 'tiger', 'value': 'El Paso, CO'}]
[{'id': '7680115', 'key': 'highway', 'type': 'regular', 'value': 'residential'}, {'id': '7680115', 'key': 'maxspeed', 'type': 'regular', 'value': '25 mph'}, {'id': '7680115', 'key': 'name', 'type': 'regular', 'value': 'Margarita Place'}, {'id': '7680115', 'key': 'cfcc', 'type': 'tiger', 'value': 'A41'}, {'id': '7680115', 'key': 'county', 'type': 'tiger', 'value': 'El Paso, CO'}, {'id': '7680115', 'key': 'name_base', 'type': 'tiger', 'value': 'Margarita'}, {'id': '7680115', 'key': 'name_type', 'type': 'tiger', 'value': 'Pl'}]
[{'id': '7680153', 'key': 'highway', 'type': 'regular', 'value': 'unclassified'}, {'id': 

[{'id': '7682136', 'key': 'highway', 'type': 'regular', 'value': 'residential'}, {'id': '7682136', 'key': 'maxspeed', 'type': 'regular', 'value': '25 mph'}, {'id': '7682136', 'key': 'name', 'type': 'regular', 'value': 'Flintlock Terrace West'}, {'id': '7682136', 'key': 'cfcc', 'type': 'tiger', 'value': 'A41'}, {'id': '7682136', 'key': 'county', 'type': 'tiger', 'value': 'El Paso, CO'}, {'id': '7682136', 'key': 'name_base', 'type': 'tiger', 'value': 'Flintlock'}, {'id': '7682136', 'key': 'name_direction_suffix', 'type': 'tiger', 'value': 'W'}, {'id': '7682136', 'key': 'name_type', 'type': 'tiger', 'value': 'Ter'}]
[{'id': '7682227', 'key': 'highway', 'type': 'regular', 'value': 'residential'}, {'id': '7682227', 'key': 'name', 'type': 'regular', 'value': 'Brunswick Drive'}, {'id': '7682227', 'key': 'cfcc', 'type': 'tiger', 'value': 'A41'}, {'id': '7682227', 'key': 'county', 'type': 'tiger', 'value': 'El Paso, CO'}, {'id': '7682227', 'key': 'name_base', 'type': 'tiger', 'value': 'Brunswic

[{'id': '7683704', 'key': 'highway', 'type': 'regular', 'value': 'residential'}, {'id': '7683704', 'key': 'name', 'type': 'regular', 'value': 'Moundsview Drive'}, {'id': '7683704', 'key': 'cfcc', 'type': 'tiger', 'value': 'A41'}, {'id': '7683704', 'key': 'county', 'type': 'tiger', 'value': 'El Paso, CO'}, {'id': '7683704', 'key': 'name_base', 'type': 'tiger', 'value': 'Moundsview'}, {'id': '7683704', 'key': 'name_type', 'type': 'tiger', 'value': 'Dr'}, {'id': '7683704', 'key': 'reviewed', 'type': 'tiger', 'value': 'no'}]
[{'id': '7683782', 'key': 'highway', 'type': 'regular', 'value': 'residential'}, {'id': '7683782', 'key': 'name', 'type': 'regular', 'value': 'Stonecrop Court'}, {'id': '7683782', 'key': 'cfcc', 'type': 'tiger', 'value': 'A41'}, {'id': '7683782', 'key': 'county', 'type': 'tiger', 'value': 'El Paso, CO'}, {'id': '7683782', 'key': 'name_base', 'type': 'tiger', 'value': 'Stonecrop'}, {'id': '7683782', 'key': 'name_type', 'type': 'tiger', 'value': 'Ct'}, {'id': '7683782', 

...

[{'id': '779743866', 'key': 'building', 'type': 'regular', 'value': 'detached'}, {'id': '779743866', 'key': 'source', 'type': 'regular', 'value': 'microsoft/BuildingFootprints'}]
[{'id': '779744345', 'key': 'building', 'type': 'regular', 'value': 'detached'}, {'id': '779744345', 'key': 'source', 'type': 'regular', 'value': 'microsoft/BuildingFootprints'}]
[{'id': '779744355', 'key': 'building', 'type': 'regular', 'value': 'detached'}, {'id': '779744355', 'key': 'source', 'type': 'regular', 'value': 'microsoft/BuildingFootprints'}]
[{'id': '779744365', 'key': 'building', 'type': 'regular', 'value': 'detached'}, {'id': '779744365', 'key': 'source', 'type': 'regular', 'value': 'microsoft/BuildingFootprints'}]
[{'id': '779744375', 'key': 'building', 'type': 'regular', 'value': 'detached'}, {'id': '779744375', 'key': 'source', 'type': 'regular', 'value': 'microsoft/BuildingFootprints'}]
[{'id': '779744385', 'key': 'building', 'type': 'regular', 'value': 'detached'}, {'id': '779744385', 'key

[{'id': '779793681', 'key': 'building', 'type': 'regular', 'value': 'detached'}, {'id': '779793681', 'key': 'source', 'type': 'regular', 'value': 'microsoft/BuildingFootprints'}]
[{'id': '779793691', 'key': 'building', 'type': 'regular', 'value': 'detached'}, {'id': '779793691', 'key': 'source', 'type': 'regular', 'value': 'microsoft/BuildingFootprints'}]
[{'id': '779793883', 'key': 'building', 'type': 'regular', 'value': 'detached'}, {'id': '779793883', 'key': 'source', 'type': 'regular', 'value': 'microsoft/BuildingFootprints'}]
[{'id': '779793893', 'key': 'building', 'type': 'regular', 'value': 'detached'}, {'id': '779793893', 'key': 'source', 'type': 'regular', 'value': 'microsoft/BuildingFootprints'}]
[{'id': '779793903', 'key': 'building', 'type': 'regular', 'value': 'detached'}, {'id': '779793903', 'key': 'source', 'type': 'regular', 'value': 'microsoft/BuildingFootprints'}]
[{'id': '779793913', 'key': 'building', 'type': 'regular', 'value': 'detached'}, {'id': '779793913', 'key

[{'id': '791961486', 'key': 'highway', 'type': 'regular', 'value': 'footway'}]
[{'id': '792747593', 'key': 'street', 'type': 'addr', 'value': 'Date Street'}, {'id': '792747593', 'key': 'building', 'type': 'regular', 'value': 'terrace'}]
[{'id': '793632033', 'key': 'bicycle', 'type': 'regular', 'value': 'designated'}, {'id': '793632033', 'key': 'foot', 'type': 'regular', 'value': 'no'}, {'id': '793632033', 'key': 'highway', 'type': 'regular', 'value': 'path'}, {'id': '793632033', 'key': 'scale', 'type': 'mtb', 'value': '4'}, {'id': '793632033', 'key': 'name', 'type': 'regular', 'value': 'Almond Butter'}, {'id': '793632033', 'key': 'oneway', 'type': 'regular', 'value': 'yes'}, {'id': '793632033', 'key': 'surface', 'type': 'regular', 'value': 'ground'}]
[{'id': '798200959', 'key': 'access', 'type': 'regular', 'value': 'private'}, {'id': '798200959', 'key': 'highway', 'type': 'regular', 'value': 'service'}, {'id': '798200959', 'key': 'service', 'type': 'regular', 'value': 'driveway'}]
[{'i

[{'id': '802345787', 'key': 'building', 'type': 'regular', 'value': 'detached'}, {'id': '802345787', 'key': 'source', 'type': 'regular', 'value': 'microsoft/BuildingFootprints'}]
[{'id': '802345969', 'key': 'building', 'type': 'regular', 'value': 'detached'}, {'id': '802345969', 'key': 'source', 'type': 'regular', 'value': 'microsoft/BuildingFootprints'}]
[{'id': '802345979', 'key': 'building', 'type': 'regular', 'value': 'detached'}, {'id': '802345979', 'key': 'source', 'type': 'regular', 'value': 'microsoft/BuildingFootprints'}]
[{'id': '802345989', 'key': 'building', 'type': 'regular', 'value': 'detached'}, {'id': '802345989', 'key': 'source', 'type': 'regular', 'value': 'microsoft/BuildingFootprints'}]
[{'id': '802345999', 'key': 'building', 'type': 'regular', 'value': 'detached'}, {'id': '802345999', 'key': 'source', 'type': 'regular', 'value': 'microsoft/BuildingFootprints'}]
[{'id': '802346009', 'key': 'building', 'type': 'regular', 'value': 'detached'}, {'id': '802346009', 'key

[{'id': '803185044', 'key': 'building', 'type': 'regular', 'value': 'yes'}, {'id': '803185044', 'key': 'source', 'type': 'regular', 'value': 'microsoft/BuildingFootprints'}]
[{'id': '803185054', 'key': 'building', 'type': 'regular', 'value': 'yes'}, {'id': '803185054', 'key': 'source', 'type': 'regular', 'value': 'microsoft/BuildingFootprints'}]
[{'id': '803185201', 'key': 'building', 'type': 'regular', 'value': 'yes'}, {'id': '803185201', 'key': 'source', 'type': 'regular', 'value': 'microsoft/BuildingFootprints'}]
[{'id': '803185211', 'key': 'building', 'type': 'regular', 'value': 'yes'}, {'id': '803185211', 'key': 'source', 'type': 'regular', 'value': 'microsoft/BuildingFootprints'}]
[{'id': '803185221', 'key': 'building', 'type': 'regular', 'value': 'yes'}, {'id': '803185221', 'key': 'source', 'type': 'regular', 'value': 'microsoft/BuildingFootprints'}]
[{'id': '803185231', 'key': 'building', 'type': 'regular', 'value': 'yes'}, {'id': '803185231', 'key': 'source', 'type': 'regular'

##Sources

https://github.com/SpecCRA/quiz_preparing_for_database_sql/blob/master/quiz_preparing_for_database_sql.py
